<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P3_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install accelerate==0.22.0
!pip install transformers==4.33.1
!pip install peft==0.5.0
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer,pipeline
from datasets import Dataset,load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
ds = load_dataset("zhengr/alpaca-chinese-dataset")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['en_input', 'en_output', 'input', 'instruction', 'output', 'en_instruction'],
        num_rows: 1000
    })
})

In [3]:
def data_process(example): #这次数据处理不进行batched，只处理单个的数据，因为label部分不太容易做
  max_length=256
  tokenized_input=tokenizer("\n".join(["User:"+example["instruction"],example["input"]]).strip()+"\nAssistant:")
  tokenized_output=tokenizer(example["output"]+tokenizer.eos_token)
  input_ids=tokenized_input["input_ids"]+tokenized_output["input_ids"]
  attention_mask=tokenized_input["attention_mask"]+tokenized_output["attention_mask"]
  labels= [-100]*len(tokenized_input["input_ids"])+tokenized_output["input_ids"]
  if len(input_ids)>max_length:
    input_ids=input_ids[:max_length]
    attention_mask=attention_mask[:max_length]
    labels=labels[:max_length]
  return {
      "input_ids":input_ids,
      "attention_mask":attention_mask,
      "labels":labels
  }
tokenized_ds=ds.map(data_process,remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [4]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'User:我们如何减少日常生活中的用水量？\nAssistant:1. 使用节水的洁具，如淋浴喷头和水龙头。\n2.使用水盆或桶收集家庭废水，例如洗碗和洗澡。\n3. 提高社区节水意识。\n4. 检查管道和灌溉系统的漏水情况，并及时修复。\n5. 淋浴时间较短，使用低流量淋浴喷头以节约用水。\n6.收集雨水并将其用于园艺或其他非饮用目的。\n7. 刷牙或洗手时关闭水龙头。\n8. 减少给草坪浇水的时间。\n9. 尽可能重复使用灰水（洗衣机、浴室水槽和淋浴的水）。\n10. 只购买节能洗碗机和洗衣机。</s>'

In [5]:
tmp=list(filter(lambda x:x != -100,tokenized_ds["train"][0]["labels"]))
tokenizer.decode(tmp)

'1. 使用节水的洁具，如淋浴喷头和水龙头。\n2.使用水盆或桶收集家庭废水，例如洗碗和洗澡。\n3. 提高社区节水意识。\n4. 检查管道和灌溉系统的漏水情况，并及时修复。\n5. 淋浴时间较短，使用低流量淋浴喷头以节约用水。\n6.收集雨水并将其用于园艺或其他非饮用目的。\n7. 刷牙或洗手时关闭水龙头。\n8. 减少给草坪浇水的时间。\n9. 尽可能重复使用灰水（洗衣机、浴室水槽和淋浴的水）。\n10. 只购买节能洗碗机和洗衣机。</s>'

In [6]:
for name, parameter in model.named_parameters():
    print(name)

transformer.word_embeddings.weight
transformer.word_embeddings_layernorm.weight
transformer.word_embeddings_layernorm.bias
transformer.h.0.input_layernorm.weight
transformer.h.0.input_layernorm.bias
transformer.h.0.self_attention.query_key_value.weight
transformer.h.0.self_attention.query_key_value.bias
transformer.h.0.self_attention.dense.weight
transformer.h.0.self_attention.dense.bias
transformer.h.0.post_attention_layernorm.weight
transformer.h.0.post_attention_layernorm.bias
transformer.h.0.mlp.dense_h_to_4h.weight
transformer.h.0.mlp.dense_h_to_4h.bias
transformer.h.0.mlp.dense_4h_to_h.weight
transformer.h.0.mlp.dense_4h_to_h.bias
transformer.h.1.input_layernorm.weight
transformer.h.1.input_layernorm.bias
transformer.h.1.self_attention.query_key_value.weight
transformer.h.1.self_attention.query_key_value.bias
transformer.h.1.self_attention.dense.weight
transformer.h.1.self_attention.dense.bias
transformer.h.1.post_attention_layernorm.weight
transformer.h.1.post_attention_layernor

In [7]:
#针对bloom，默认的target_modules就是query_key_value
#target_modules：想要进行lora微调的部分，modules_to_save：除了进行lora以外、还想进行训练和保存的部分，这些部分可以通过model.named_parameters获取
config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["query_key_value","dense"], modules_to_save=["word_embeddings"])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['query_key_value', 'dense'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [8]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",low_cpu_mem_usage=True)
model = get_peft_model(model, config)

In [9]:
for name, parameter in model.named_parameters():
    print(name)
#注意看，这时候的query_key_value层和dense层都多了loraA和loraB（两个lora低秩矩阵矩阵A*B构成参数更新的deltaW

base_model.model.transformer.word_embeddings.original_module.weight
base_model.model.transformer.word_embeddings.modules_to_save.default.weight
base_model.model.transformer.word_embeddings_layernorm.weight
base_model.model.transformer.word_embeddings_layernorm.bias
base_model.model.transformer.h.0.input_layernorm.weight
base_model.model.transformer.h.0.input_layernorm.bias
base_model.model.transformer.h.0.self_attention.query_key_value.weight
base_model.model.transformer.h.0.self_attention.query_key_value.bias
base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight
base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight
base_model.model.transformer.h.0.self_attention.dense.weight
base_model.model.transformer.h.0.self_attention.dense.bias
base_model.model.transformer.h.0.self_attention.dense.lora_A.default.weight
base_model.model.transformer.h.0.self_attention.dense.lora_B.default.weight
base_model.model.transformer.h.0.post_at

In [10]:
#针对bloom，默认的target_modules就是query_key_value
#target_modules也支持正则匹配
config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=".*\.1.*query_key_value", modules_to_save=["word_embeddings"])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules='.*\\.1.*query_key_value', lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [11]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",low_cpu_mem_usage=True)

model = get_peft_model(model, config)
for name, parameter in model.named_parameters():
    print(name)
#只对既包含1又包含query_key_value的部分进行lora
#可以通过print(model.print_trainable_parameters())，对添加了modules_to_save=["word_embeddings"]和没添加modules_to_save=["word_embeddings"]的model进行对比，来验证是否成功添加了需要训练的Module（参数量也变多）

base_model.model.transformer.word_embeddings.original_module.weight
base_model.model.transformer.word_embeddings.modules_to_save.default.weight
base_model.model.transformer.word_embeddings_layernorm.weight
base_model.model.transformer.word_embeddings_layernorm.bias
base_model.model.transformer.h.0.input_layernorm.weight
base_model.model.transformer.h.0.input_layernorm.bias
base_model.model.transformer.h.0.self_attention.query_key_value.weight
base_model.model.transformer.h.0.self_attention.query_key_value.bias
base_model.model.transformer.h.0.self_attention.dense.weight
base_model.model.transformer.h.0.self_attention.dense.bias
base_model.model.transformer.h.0.post_attention_layernorm.weight
base_model.model.transformer.h.0.post_attention_layernorm.bias
base_model.model.transformer.h.0.mlp.dense_h_to_4h.weight
base_model.model.transformer.h.0.mlp.dense_h_to_4h.bias
base_model.model.transformer.h.0.mlp.dense_4h_to_h.weight
base_model.model.transformer.h.0.mlp.dense_4h_to_h.bias
base_mod

In [12]:
args = TrainingArguments(
    output_dir="./gen_chatbot_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=5,
    num_train_epochs=1,
    save_steps=5
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,2.925400
10,3.002600


KeyboardInterrupt: 

In [ ]:
# args = TrainingArguments(
#     output_dir="./chatbot",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8,
#     logging_steps=10,
#     num_train_epochs=1
# )

In [16]:
# model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
p_model = PeftModel.from_pretrained(model, model_id="/content/drive/MyDrive/nlp/gen_chatbot_lora/checkpoint-10")
p_model

p_model = p_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(p_model.device)
tokenizer.decode(p_model.generate(**ipt,max_length=128)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 考试有哪些技巧？\n1. 答题时，要保持冷静，不要情绪化。 2. 答题时，要保持清晰的思路，不要胡思乱想。 3. 答题时，要保持准确的表达，不要胡言乱语。 4. 答题时，要保持准确的表达，不要胡言乱语。 5. 答题时，要保持准确的表达，不要胡言乱语。 6. 答题时，要保持准确的表达，不要胡言乱语。'

In [18]:
p_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): BloomForCausalLM(
          (transformer): BloomModel(
            (word_embeddings): Embedding(46145, 2048)
            (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (h): ModuleList(
              (0-23): 24 x BloomBlock(
                (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                (self_attention): BloomAttention(
                  (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
                  (dense): Linear(in_features=2048, out_features=2048, bias=True)
                  (attention_dropout): Dropout(p=0.0, inplace=False)
                )
                (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                (mlp): BloomMLP(
                  (dense_h_to_4h): Linear(in_features=2048, ou

In [19]:
#模型合并并保存：
merge_model = p_model.merge_and_unload()
merge_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(46145, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
              (dense): Linear(in_features=2048, out_features=2048, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (mlp): BloomMLP(
              (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
              (gelu_impl): BloomGelu()
              (dense_4h_to_h): Linear(in_features=8192, out

In [22]:
merge_model.save_pretrained("/content/drive/MyDrive/nlp/gen_chatbot_lora/mergemodel")
#下次调用直接：
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/nlp/gen_chatbot_lora/mergemodel", low_cpu_mem_usage=True) #不需要再使用peft了

In [23]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
      (1): BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwi